## Step 1: Mount Google Drive & Copy Files

In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')
print("✓ Google Drive mounted")

# List your Drive to find the repo
!ls /content/drive/MyDrive/ | head -20

### Option A: Copy from Drive (if repo is there)

In [ ]:
# Adjust path to your Drive folder
!cp -r /content/drive/MyDrive/enhancedlibcem /content/ 2>/dev/null || echo "⚠ enhancedlibcem not found in Drive root. Adjust path above."
!ls -la /content/enhancedlibcem/improved_lidecm.pt 2>/dev/null || echo "⚠ Checkpoint not found in copied folder"

### Option B: Clone from GitHub (if repo is public)

In [ ]:
# If repo is on GitHub, clone it:
# !git clone https://github.com/your-user/enhancedlibcem.git /content/enhancedlibcem
# !ls -la /content/enhancedlibcem/

# For now, check if folder already exists
!test -d /content/enhancedlibcem && echo "✓ enhancedlibcem folder exists" || echo "⚠ Folder not yet copied. Use Option A or B above."

### Option C: Upload checkpoint separately if needed

In [ ]:
# If checkpoint is in a specific Drive folder, copy it:
!cp /content/drive/MyDrive/path/to/improved_lidecm.pt /content/enhancedlibcem/ 2>/dev/null || echo "Adjust path or use Drive UI to upload"
!ls -lh /content/enhancedlibcem/improved_lidecm.pt 2>/dev/null || echo "⚠ Checkpoint still missing"

## Step 2: Install Dependencies

In [ ]:
# Install PyTorch with CUDA support (already available in Colab, but ensure correct version)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
print("✓ PyTorch installed")

In [ ]:
# Install CLIP and other dependencies
!pip install -q git+https://github.com/openai/CLIP.git numpy matplotlib scikit-image scipy numba pillow tqdm
print("✓ All dependencies installed")

## Step 3: Verify Setup

In [ ]:
import torch
import sys
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

# Check checkpoint
checkpoint_path = Path('/content/enhancedlibcem/improved_lidecm.pt')
if checkpoint_path.exists():
    size_mb = checkpoint_path.stat().st_size / (1024**2)
    print(f"✓ Checkpoint found: {size_mb:.1f} MB")
else:
    print("⚠ Checkpoint NOT found at /content/enhancedlibcem/improved_lidecm.pt")
    print("Please upload improved_lidecm.pt to your Drive or /content directly")

## Step 4: Run Quick Test (Diagnostics)

In [ ]:
%cd /content/enhancedlibcem
!python quick_test.py

In [ ]:
# Display diagnostics
with open('/content/enhancedlibcem/quick_test_outputs/diagnostics.txt', 'r') as f:
    print(f.read())

In [ ]:
# Display generated images
from IPython.display import Image, display
import os

print("=== Original Images ===")
if os.path.exists('/content/enhancedlibcem/quick_test_outputs/original_0.png'):
    display(Image('/content/enhancedlibcem/quick_test_outputs/original_0.png'))

print("\n=== Reconstructed Images ===")
if os.path.exists('/content/enhancedlibcem/quick_test_outputs/reconstructed_0.png'):
    display(Image('/content/enhancedlibcem/quick_test_outputs/reconstructed_0.png'))

## Step 5: Run Inference with Pre-trained Checkpoint

In [ ]:
%cd /content/enhancedlibcem
!python inference_from_checkpoint.py --checkpoint improved_lidecm.pt --outdir /content/inference_out

In [ ]:
# Display inference results
from IPython.display import Image, display
import os

result_path = '/content/inference_out/inference_results.png'
if os.path.exists(result_path):
    print("✓ Inference Results (5 Generated Images):")
    display(Image(result_path))
else:
    print("⚠ inference_results.png not found. Check checkpoint was loaded correctly.")

## Step 6: Save Results to Drive

In [ ]:
import shutil
from pathlib import Path

# Create results folder in Drive
results_dir = Path('/content/drive/MyDrive/enhancedlibcem_results')
results_dir.mkdir(exist_ok=True)

# Copy quick_test outputs
src = Path('/content/enhancedlibcem/quick_test_outputs')
if src.exists():
    shutil.copytree(src, results_dir / 'quick_test_outputs', dirs_exist_ok=True)
    print("✓ quick_test_outputs copied to Drive")

# Copy inference outputs
src = Path('/content/inference_out')
if src.exists():
    shutil.copytree(src, results_dir / 'inference_out', dirs_exist_ok=True)
    print("✓ inference_out copied to Drive")

print(f"\n✅ All results saved to: /MyDrive/enhancedlibcem_results")

## Summary

In [ ]:
import os
from pathlib import Path

print("=" * 70)
print("COLAB EXECUTION SUMMARY")
print("=" * 70)

# Check outputs
quick_test_dir = Path('/content/enhancedlibcem/quick_test_outputs')
inference_dir = Path('/content/inference_out')

if quick_test_dir.exists():
    files = list(quick_test_dir.glob('*'))
    print(f"\n✓ quick_test_outputs ({len(files)} files):")
    for f in files:
        size = f.stat().st_size if f.is_file() else 'dir'
        print(f"  - {f.name} ({size} bytes)")

if inference_dir.exists():
    files = list(inference_dir.glob('*'))
    print(f"\n✓ inference_out ({len(files)} files):")
    for f in files:
        size = f.stat().st_size if f.is_file() else 'dir'
        print(f"  - {f.name} ({size} bytes)")

print("\n" + "=" * 70)
print("✅ Colab execution completed successfully!")
print("=" * 70)